# import libraries 

In [1]:
#import libraries 
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# import seaborn as sns
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer

import torch


/home/suyash/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [3]:
#load the data from reviews_Movies_and_TV.json file using for loop to iterate through 40,000 sentences in a loop and append it to a list
data = []
with open('reviews_Movies_and_TV.json') as f:
    count = 0 
    for line in f:
        count = count + 1
        data.append(json.loads(line))
        if count == 80000:
            break
print(len(data))

80000


In [4]:
# for i in range(5):
# print(data[100000]['reviewText'])


# extract the review text from data and append it to a list
review_text = []
for i in range(len(data)):
    review_text.append(data[i]['reviewText'])
print((review_text[1000:1531]))

['One of the few movies kids can watch without mom worrying about inappropriate scenes or information. Perfect for the holidays.', 'Love the old clay-mation movies. Glad they are still around and for such a low price. Kids today and tomorrow will love these; they truly are timeless.', "Still my favorite after all these years, it's nice to have a special that can present the HO-HO-HO of the holidays and find a way to tie Santa in to the spiritual aspects of the Christmas season.  Wonderful animation, humor and songs-I am glad I finally got this one on DVD.", 'Santa Claus is coming to town is one of the best christmas movies ever.  I loved it as a child and bought it for my son to experience the magic of the claymation, the songs, and to learn why Santa Claus does the things he  does.  It is good to see something that you can watch over and over again  without getting bored.', 'This was a Christmas gift for my son, he loved it.  Thank you!!  Nothing like a good ole classic to brighten yo

In [6]:
#tokenize the review text using into sentences
sentences = []
for i in range(len(review_text)):
    sentences.append(word_tokenize(review_text[i]))
print(sentences[1000:1531])




[['One', 'of', 'the', 'few', 'movies', 'kids', 'can', 'watch', 'without', 'mom', 'worrying', 'about', 'inappropriate', 'scenes', 'or', 'information', '.', 'Perfect', 'for', 'the', 'holidays', '.'], ['Love', 'the', 'old', 'clay-mation', 'movies', '.', 'Glad', 'they', 'are', 'still', 'around', 'and', 'for', 'such', 'a', 'low', 'price', '.', 'Kids', 'today', 'and', 'tomorrow', 'will', 'love', 'these', ';', 'they', 'truly', 'are', 'timeless', '.'], ['Still', 'my', 'favorite', 'after', 'all', 'these', 'years', ',', 'it', "'s", 'nice', 'to', 'have', 'a', 'special', 'that', 'can', 'present', 'the', 'HO-HO-HO', 'of', 'the', 'holidays', 'and', 'find', 'a', 'way', 'to', 'tie', 'Santa', 'in', 'to', 'the', 'spiritual', 'aspects', 'of', 'the', 'Christmas', 'season', '.', 'Wonderful', 'animation', ',', 'humor', 'and', 'songs-I', 'am', 'glad', 'I', 'finally', 'got', 'this', 'one', 'on', 'DVD', '.'], ['Santa', 'Claus', 'is', 'coming', 'to', 'town', 'is', 'one', 'of', 'the', 'best', 'christmas', 'movie

In [8]:
# #remove punctuations from the sentences using regular expression
# for i in range(len(sentences)):
#     sentences[i] = [re.sub(r'[^\w\s]', '', word) for word in sentences[i]]
# print(sentences[1000:1531])

'This'

In [10]:
#remove the stop words and punctuations from the sentences
stop_words = set(stopwords.words('english'))
for i in range(len(sentences)):
    sentences[i] = [re.sub(r'[^\w\s]', '', word) for word in sentences[i] if word not in stop_words and word != '']
    # sentences[i] = [word for word in sentences[i] if word not in stop_words]
print(sentences[1000:1531])


[['One', 'movies', 'kids', 'watch', 'without', 'mom', 'worrying', 'inappropriate', 'scenes', 'information', 'Perfect', 'holidays'], ['Love', 'old', 'claymation', 'movies', 'Glad', 'still', 'around', 'low', 'price', 'Kids', 'today', 'tomorrow', 'love', 'truly', 'timeless'], ['Still', 'favorite', 'years', 'nice', 'special', 'present', 'HOHOHO', 'holidays', 'find', 'way', 'tie', 'Santa', 'spiritual', 'aspects', 'Christmas', 'season', 'Wonderful', 'animation', 'humor', 'songsI', 'glad', 'I', 'finally', 'got', 'one', 'DVD'], ['Santa', 'Claus', 'coming', 'town', 'one', 'best', 'christmas', 'movies', 'ever', 'I', 'loved', 'child', 'bought', 'son', 'experience', 'magic', 'claymation', 'songs', 'learn', 'Santa', 'Claus', 'things', 'It', 'good', 'see', 'something', 'watch', 'without', 'getting', 'bored'], ['This', 'Christmas', 'gift', 'son', 'loved', 'Thank', 'Nothing', 'like', 'good', 'ole', 'classic', 'brighten', 'day'], ['What', 'I', 'say', 'wonderful', 'Christmas', 'movie', 'family', 'Arrive

In [5]:
# #cleaning the data

# #convert into sentences 
# sentences = []

# #remove all special characters except @,#,.,?!
# for text in review_text:
#     text = re.sub(r'[^\@\#\.\w\?\!\s]', '', text)
#     #remove all extra spaces and newlines
#     text = re.sub(r'\s+', ' ', text)
#     text = re.sub(r'\n*', '', text)
#     text = re.sub(r'\.+', '.', text)

#     #remove . from  abbrevations 
#     f = re.compile(r'([A-Z]([a-z]){,2}\.)')
#     a = re.findall(f, text)
#     if(a):
#         abbrevations_set = set((list(zip(*a))[0]))

#         for word in abbrevations_set:
#             text = text.replace(word, word.strip('.'))

#     # Replace hashtags with <HASHTAG>
#     text = re.sub(r'#\w+', '<HASHTAG>', text)

#     #replace @ with <MENTION>
    
#     text = re.sub(r'@\w+', '<MENTION>', text)


#     #replace http with <URL>
#     text = re.sub(r'http\S+', '<URL>', text)
    
#     #replace www with <URL> 
#     text = re.sub(r'www\S+', '<URL>', text)

#     #replace Date with <DATE>
#     text = re.sub(r'\d{1,2}\/\d{1,2}\/\d{2,4}', '<DATE>', text)

#     #replace Time with <TIME>
#     text = re.sub(r'\d{1,2}:\d{2}', '<TIME>', text)

#     #replace percentage with <PERCENT>
#     text = re.sub(r'\d{1,3}\%', '<PERCENT>', text)

#     text = re.sub(r'\b\d+\b', '<NUMBER>', text)


#      #replace _ with space
#     text = re.sub(r'_', '', text)
#     #change to lowercase
#     text = text.lower()   

#     # Split the text into sentences 
#     sentences.append(re.split(r'[.!?]+', text))


In [6]:
# #remove empty sentences and add all the sentences to a list
# sentences_list = []
# for sentence in sentences:
#     for s in sentence:
#         if s != '':
#             sentences_list.append(s)
# print((sentences_list[12:20]))


[' another disappointment is the so so transfer quality to dvd perhaps ive been spoiled by hd and tivo but the older shows ive recorded have had better playback quality than the episodes on the dvds it is in the old 480p but the quality of the transfer to dvd is dark and about the same quality as your average old vhs tape not dvd quality', ' i get the impression food network got cheap and subbed out the dvd transfer to the lowest bidder china', ' and it shows', 'i could watch giada read the dictionary and her cooking is really first rate', ' but thats all you get is watching giada', ' thank god she is easy to understand and her recipes are easy to follow', ' but to get consistent results on some of the dishes you should search the web and find the hard copies', ' while giada herself is great this set is a waste of money']


In [7]:
#save the sentences to a file in json format
with open('sentences.json', 'w') as f:
    json.dump(sentences, f)

In [8]:
#load the sentences from the file sentences.json
with open('sentences.json') as f:
    saved_sentences = json.load(f)



In [9]:
print(len(saved_sentences))
print(saved_sentences[1000:1531])

600528
[' we purchased this as a christmas gift for her she just loves it', ' thank you', 'i liked this production giving the possible american point of view', ' weve all seen the original christmas carol but id highly recommend this to anyone', ' we watched this with our grandchildren whose interest was kept the entire length', 'i thought his was a great twist on a christmas carol that put it in a more relevant perspective for these times', 'as we were picking christmas movies this year i remembered seeing one with henry winkler', ' took me a while to find it because i didnt recall the title correctly', ' told my wife she would really like it', ' was not disappointed', ' a very good christmas classic in my book', ' winkler is terrific in this role', 'great unless you are just tired of christmas movies along charles dickens christmas carol theme', ' it is certainly one of the better ones based on dickens book and had been unavailable for purchase for to many years', 'i collect movie ve

In [10]:
tokens = []
for i in range(len(saved_sentences)):
        #remove punctuations and special characters
        sentence = saved_sentences[i]
        sentence = re.sub(r'[^\w<>]', ' ', sentence)
        sentence = re.sub(r'\s+', ' ', sentence.strip())        
        saved_sentences[i] = sentence
        words = sentence.split()
        # words.insert(0, '<s>')
        # words.append('</s>')

        tokens.extend(words)

# #remove a empty token from the list
# tokens = [token for token in tokens if len(token) > 0]

#replace words with <UNK> if they occur less than 2 times in the corpus

# freq = {}
# k=0
# for i in range(len(tokens)):
#         if tokens[i] in freq:
#             freq[tokens[i]] += 1
#         else:
#             freq[tokens[i]] = 1

# # for i in range(len(tokens)):
# #         if freq[tokens[i]] < 3:
# #             k+=1
# #             # tokens[i] = '<UNK>'


In [11]:
print(tokens[0])    


this
